In [ ]:
# coding:utf-8      
# version:python3.6
# author:Ziyu Liu
# collect street view data from BaiduMap
# Guidance:http://lbsyun.baidu.com/index.php?title=viewstatic


import requests
import os
import time
angles = [0,90,180,270]
pitchs = [45,90]

#BaiduAK.txt is the file of Your Baidu AKs
with open(r'./BaiduAK.txt', 'r') as apis:
    apilines = apis.readlines()
    #print("read")

# Baidu API request
class BaiduAPI(object):
    def __init__(self):
        # Your Baidu AK
        self.ak = apilines[0].strip('\t\r\n')
        self.keycount = 1
        #print("set")

    # Each search request

    def search_photo(self, longitude, latitude,ID):
        print("function")
        data = {
            "ak": self.ak,
            "height": "360",
            "width": "1024",
            "location": "{0},{1}".format(latitude,longitude),
            "fov": "360",
            "pitch": "0",
            "heading": "0",
            "coordtype":"wgs84ll",
        }
        try:
            # Download pictures
            r = requests.get("http://api.map.baidu.com/panorama/v2?", params=data)
            print("request")
            print (self.ak)
            print(r.status_code)
            savepath="./{0}_{1}_{2}.jpg".format(ID,longitude,latitude)
            open(savepath, 'wb').write(r.content)
            filesize=os.stat(savepath).st_size
            if r.status_code == 200:
                os.remove(savepath)
                return filesize
            #print (params)
            print ("--------filesize-------------")
            print (filesize)
            if filesize==101:
                os.remove(savepath)
                return filesize
            while filesize == 59 or filesize== 41 or filesize==77 or filesize==89 or filesize==47:
                os.remove(savepath)
#                 print ("-----------------keychange----------------------\n")
                print (self.keycount)
                #time.sleep(3)
                print (self.ak)
                print (ID)
                if int(self.keycount) < len(apilines):
                    print ("panduan!!!!!!!!!!!!!!!!!!!")
                    #time.sleep(3)
                    self.keycount = int(self.keycount) + 1
                    self.ak = apilines[self.keycount-1].strip('\t\r\n')
                    del apilines[0]
                print ("current key count is ")
                print (self.keycount)
                print (self.ak)
                time.sleep(3)
                print ("--------request the fail Try Again---------")
                filesize=self.search_photo(longitude,latitude,ID)
                print (self.ak)
                print("---------------------------filesize----------------")
                print (filesize)
                print ("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n")
                return filesize
        except Exception as e:
            open("BSV.txt", 'a').writelines(e)



if __name__ == '__main__':
	# Read data from csv
	with open(r'./baishan_bsvpoint_2.csv', 'r') as data:
		lines = data.readlines()
		for i in range(20521,20522):
			# Get coordinates
			latitude = lines[i].split(',')[3]
			longitude = lines[i].split(',')[4]
			ID = lines[i].split(',')[0]
			longitude=longitude.strip('\t\r\n')
			# Get pictures
			baidu = BaiduAPI()
			print(longitude, latitude,ID)
			time.sleep(2)
			baidu.search_photo(longitude,latitude,ID)
				



In [94]:
# coding:utf-8      
# version:python3.6
# author:Ziyu Liu
# collect street view data from BaiduMap
# Guidance:http://lbsyun.baidu.com/index.php?title=viewstatic


import requests
import os
import time
import json
angles = [0,90,180,270]
pitchs = [45,90]

#BaiduAK.txt is the file of Your Baidu AKs
with open(r'/home/user/PV/BJBSV/BaiduAK2.txt', 'r') as apis:
    apilines = apis.readlines()
    #print("read")

# Baidu API request
class BaiduAPI(object):
    def __init__(self):
        # Your Baidu AK
        self.ak = apilines[0].strip('\t\r\n')
        self.keycount = 1
        self.imgcount = 0
        #print("set")

    # Each search request
    def search_photo(self, longitude, latitude,ID):
        filesize = 0
        data = {
            "ak": self.ak,
            "height": "360",
            "width": "1024",
            "location": "{0},{1}".format(longitude,latitude),
            "fov": "360",
            "pitch": "0",
            "heading": "0",
            "coordtype":"wgs84ll",
        }
        try:
            # Download pictures
            time.sleep(0.3)
            r = requests.get("http://api.map.baidu.com/panorama/v2?", params=data)
            r.encoding='utf-8'
            print(self.ak)
            savepath="./SYBSV/{0}_{1}_{2}.jpg".format(ID,longitude,latitude)
            open(savepath, 'wb').write(r.content)
            filesize=os.stat(savepath).st_size
            # 301 Permanent quota exceeds limit, rest
            if filesize > 200 and self.imgcount <= 80:
                self.imgcount += 1
                print(self.imgcount)
                return filesize
            else:
                data = r.json()
                print(data)
                status = data["status"] 
                print (status)
                # 301 Permanent quota exceeds limit, restrict access
                # 302 Day quota exceeded, restrict access
                if status == "301" or status == "302" or status == "201" or self.imgcount > 80:
                    print("301/2")
                    os.remove(savepath)
                    print ("-----------------keychange----------------------\n")
                    print (self.keycount,self.ak,ID)
                    #time.sleep(3)
                    if int(self.keycount) < len(apilines):
                        remainnum = int(len(apilines))-int(self.keycount)
                        print ("We still have %d AKs"%remainnum)
                        time.sleep(2)
                        #self.keycount = int(self.keycount) + 1
                        self.ak = apilines[self.keycount-1].strip('\t\r\n')
                        self.imgcount = 0
                        del apilines[0]
                        print ("current key count is %s, no. %d"%(self.ak,self.keycount))
                        print ("--------Request Again---------")
                        filesize = self.search_photo(longitude,latitude,ID)
                    else:
                        print("ALL AKs HAVE DONE and LAST ID is %s"%ID)
                        filesize = 0
                        sys.exit(1)
                # 401 The current concurrency has exceeded the agreed concurrency quota, restrict access
                elif status == "401" :
                    print('401')
                    os.remove(savepath)
                    #time.sleep(1)
                    print ("--------request failed Try Again---------")
                    filesize=self.search_photo(longitude,latitude,ID)
                else:
                    os.remove(savepath)
                    filesize = 0
        except Exception as e:
            # open("BSV.txt", 'a').writelines(e)
            print(e)
        print ("--------filesize  %s-------------"%filesize)
        return filesize

            
if __name__ == '__main__':
	# Read data from csv
	with open(r'./SY_Point_BSV/SYAllPoints.csv', 'r') as data:
		lines = data.readlines()
		for i in range(15475,len(lines)):
			# Get coordinates
			longitude = lines[i].split(',')[3]
			latitude = lines[i].split(',')[4]
			ID = lines[i].split(',')[0]
			latitude = latitude.strip('\t\r\n')
			# Get pictures
			baidu = BaiduAPI()
			print(longitude, latitude,ID)
			#time.sleep(2)
			baidu.search_photo(longitude,latitude,ID)

41.79458795750 126.89539695200 70038
96Vld0tV9sNcEhDi4tkQlpwyDXpoPbzZ
{'message': '天配额超限，限制访问', 'status': '302'}
302
301/2
-----------------keychange----------------------

1 96Vld0tV9sNcEhDi4tkQlpwyDXpoPbzZ 70038
We still have 16 AKs
current key count is 96Vld0tV9sNcEhDi4tkQlpwyDXpoPbzZ, no. 1
--------Request Again---------
96Vld0tV9sNcEhDi4tkQlpwyDXpoPbzZ
{'message': '天配额超限，限制访问', 'status': '302'}
302
301/2
-----------------keychange----------------------

1 96Vld0tV9sNcEhDi4tkQlpwyDXpoPbzZ 70038
We still have 15 AKs
current key count is , no. 1
--------Request Again---------

{'message': 'AK 参数无效', 'status': '500'}
500
--------filesize  0-------------
--------filesize  0-------------
--------filesize  0-------------
41.79502550550 126.89515497700 70039
N6p4qtXjeg5AB8kFxuVVKYmXlfQW5gtj
{'message': 'APP被用户自己禁用，请在控制台解禁', 'status': '201'}
201
301/2
-----------------keychange----------------------

1 N6p4qtXjeg5AB8kFxuVVKYmXlfQW5gtj 70039
We still have 14 AKs
current key count is N6p4qtX

KeyboardInterrupt: 

In [ ]:
# 

In [ ]:
# coding:utf-8      
# version:python3.6
# author:Ziyu Liu
# collect street view data from BaiduMap
# Guidance:http://lbsyun.baidu.com/index.php?title=viewstatic


import sys
import requests
import os
import time
import json
angles = [0,90,180,270]
pitchs = [45,90]

#BaiduAK.txt is the file of Your Baidu AKs
with open(r'/home/user/PV/BJBSV/BaiduAK.txt', 'r') as apis:
    apilines = apis.readlines()
    #print("read")

# Baidu API request
class BaiduAPI(object):
    def __init__(self):
        # Your Baidu AK
        self.ak = apilines[0].strip('\t\r\n')
        self.keycount = 1
        #print("set")

    # Each search request
    def search_photo(self, longitude, latitude,ID):
        filesize = 0
        data = {
            "ak": self.ak,
            "height": "360",
            "width": "1024",
            "location": "{0},{1}".format(latitude,longitude),
            "fov": "360",
            "pitch": "0",
            "heading": "0",
            "coordtype":"wgs84ll",
        }
        try:
            # Download pictures
            r = requests.get("http://api.map.baidu.com/panorama/v2?", params=data)
            r.encoding='utf-8'
            print(self.ak)
            savepath="./SYBSV/{0}_{1}_{2}.jpg".format(ID,longitude,latitude)
            open(savepath, 'wb').write(r.content)
            filesize=os.stat(savepath).st_size
            # 301 Permanent quota exceeds limit, rest
            if filesize > 200:
                return filesize
            else:
                data = r.json()
                print(data)
                status = data["status"]
                print (status)
                # 301 Permanent quota exceeds limit, restrict access
                # 302 Day quota exceeded, restrict access
                if status == "301" or status == "302" or status == "201":
                    print("301/2")
                    os.remove(savepath)
                    print ("--------request failed Try Again---------")
                    filesize=self.search_photo(longitude,latitude,ID)
                else:
                    os.remove(savepath)
                    filesize = 0
        except Exception as e:
            # open("BSV.txt", 'a').writelines(e)
            print(e)
        print ("--------filesize  %s-------------"%filesize)
        return filesize


if __name__ == '__main__':
    # Read data from csv
    with open(r'./SY_Point_BSV/SYAllPoints.csv', 'r') as data:
        lines = data.readlines()
        for i in range(6684,len(lines)):
            # Get coordinates
            longitude = lines[i].split(',')[3]
            latitude = lines[i].split(',')[4]
            ID = lines[i].split(',')[0]
            latitude=latitude.strip('\t\r\n')
            # Get pictures
            baidu = BaiduAPI()
            print(longitude, latitude,ID)
            #time.sleep(2)
            baidu.search_photo(longitude,latitude,ID)
                                                                          

In [80]:
# coding:utf-8      
# version:python3.6
# author:Ziyu Liu
# collect street view data from BaiduMap
# Guidance:http://lbsyun.baidu.com/index.php?title=viewstatic


import requests
import os
import time
angles = [0,90,180,270]
pitchs = [45,90]

#BaiduAK.txt is the file of Your Baidu AKs
with open(r'./BaiduAK.txt', 'r') as apis:
    apilines = apis.readlines()
    #print("read")

# Baidu API request
class BaiduAPI(object):
    def __init__(self):
        # Your Baidu AK
        self.ak = apilines[0].strip('\t\r\n')
        self.keycount = 1
        #print("set")

    # Each search request

    def search_photo(self, longitude, latitude,ID):
        print("function")
        data = {
            "ak": self.ak,
            "height": "360",
            "width": "1024",
            "location": "{0},{1}".format(latitude,longitude),
            "fov": "360",
            "pitch": "0",
            "heading": "0",
            "coordtype":"wgs84ll",
        }
        try:
            # Download pictures
            r = requests.get("http://api.map.baidu.com/panorama/v2?", params=data)
            print("request")
            print (self.ak)
            print(r.status)
            savepath="./{0}_{1}_{2}.jpg".format(ID,longitude,latitude)
            open(savepath, 'wb').write(r.content)
        except Exception as e:
            # open("BSV.txt", 'a').writelines(e)
            print("exception")
        return 0

if __name__ == '__main__':
    # Read data from csv
    with open(r'./baishan_bsvpoint_2.csv', 'r') as data:
        lines = data.readlines()
        for i in range(20034,20035):
            # Get coordinates
            latitude = lines[i].split(',')[3]
            longitude = lines[i].split(',')[4]
            ID = lines[i].split(',')[0]
            longitude=longitude.strip('\t\r\n')
            # Get pictures
            baidu = BaiduAPI()
            print(longitude, latitude,ID)
            time.sleep(2)
            baidu.search_photo(longitude,latitude,ID)



41.79502255570 126.89418814600 70033
function
request

exception
